# 因子研究：资金费率因子

## 研究假设

**假设**：永续合约的资金费率反映市场多空失衡，极端费率（>0.1%/8h）预示价格反转。

**理论依据**：
- 资金费率是永续合约特有的机制，用于锚定现货价格
- 高正费率 → 多头过热 → 可能回调
- 高负费率 → 空头过热 → 可能反弹

**验证目标**：
1. IC（信息系数）> 0.05
2. t值 > 2（统计显著性）
3. 样本外测试稳定

---

In [ ]:
# 导入依赖
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 数据加载

In [ ]:
# 加载OHLCV数据
data_path = Path("../../qlib_data/okx/BTC_USDT/15m.parquet")
df = pd.read_parquet(data_path)

print(f"数据形状：{df.shape}")
print(f"时间范围：{df.index.min()} 至 {df.index.max()}")
df.head()

## 2. 因子计算

注意：资金费率数据需要单独获取（通过交易所API）。
这里先用简化版本：基于价格偏离度估算资金费率。

In [ ]:
# TODO: 实际项目中需要从交易所API获取真实资金费率
# 这里用简化版本：价格相对MA的偏离度
df['ma_24h'] = df['close'].rolling(window=96).mean()  # 24小时均线（96个15分钟）
df['price_deviation'] = (df['close'] - df['ma_24h']) / df['ma_24h']

# 模拟资金费率（实际应该用真实数据）
df['funding_rate_proxy'] = df['price_deviation'] * 0.01  # 简化版本

df[['close', 'ma_24h', 'price_deviation', 'funding_rate_proxy']].tail(10)

## 3. 计算未来收益（标签）

In [ ]:
# 计算未来8小时收益（32个15分钟K线）
df['forward_return_8h'] = df['close'].shift(-32) / df['close'] - 1

# 移除缺失值
df_clean = df.dropna()

print(f"有效样本数：{len(df_clean)}")

## 4. IC分析（信息系数）

In [ ]:
# 计算IC（因子与未来收益的相关性）
ic = df_clean['funding_rate_proxy'].corr(df_clean['forward_return_8h'])

print(f"IC（信息系数）：{ic:.4f}")
print(f"目标：IC > 0.05")
print(f"结果：{'✓ 通过' if abs(ic) > 0.05 else '✗ 未通过'}")

## 5. 分位数分析

In [ ]:
# 将因子分为5个分位数
df_clean['factor_quantile'] = pd.qcut(df_clean['funding_rate_proxy'], q=5, labels=False, duplicates='drop')

# 计算每个分位数的平均收益
quantile_returns = df_clean.groupby('factor_quantile')['forward_return_8h'].mean()

print("\n各分位数平均收益：")
print(quantile_returns)

# 可视化
quantile_returns.plot(kind='bar', title='因子分位数 vs 未来收益')
plt.xlabel('因子分位数（0=最低，4=最高）')
plt.ylabel('平均8小时收益')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

## 6. 样本外测试

In [ ]:
# 划分训练集和测试集（70% / 30%）
split_idx = int(len(df_clean) * 0.7)
train = df_clean.iloc[:split_idx]
test = df_clean.iloc[split_idx:]

# 计算训练集和测试集的IC
ic_train = train['funding_rate_proxy'].corr(train['forward_return_8h'])
ic_test = test['funding_rate_proxy'].corr(test['forward_return_8h'])

print(f"训练集 IC：{ic_train:.4f}")
print(f"测试集 IC：{ic_test:.4f}")
print(f"IC衰减：{(ic_train - ic_test) / ic_train * 100:.1f}%")

## 7. 结论

**待填写**：
- [ ] IC是否满足要求（>0.05）？
- [ ] 样本外测试是否稳定？
- [ ] 是否需要改进因子定义？
- [ ] 下一步行动计划？

---

**重要提醒**：
1. 本Notebook使用的是简化版资金费率（基于价格偏离度），实际应使用交易所API获取真实资金费率
2. 需要在多个市场周期（牛市、熊市、震荡）中验证因子稳定性
3. 只有通过严格验证的因子才能进入策略执行层